In [16]:
import pandas as pd
import re
from datetime import datetime

# --- Config ---
INPUT_FILE = "/home/veronica/Desktop/dod-unprocessed-data/Facebook_Data_batch_eg_20251006.csv"
OUTPUT_VALID = "/home/veronica/Desktop/dod-unprocessed-data/process_file.csv"
OUTPUT_INVALID = "/home/veronica/Desktop/dod-unprocessed-data/invalid_rows.csv"

# --- Regex patterns ---
EMAIL_REGEX = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
PHONE_REGEX = re.compile(r"^\+?[\d\s()-]{6,}$")

def is_valid_email(value):
    return isinstance(value, str) and EMAIL_REGEX.match(value.strip())

def is_valid_phone(value):
    if not isinstance(value, str):
        value = str(value)
    value = value.strip()
    return bool(PHONE_REGEX.match(value))



In [17]:
# 🧩 Cell 2: Load and Inspect Input Data
df = pd.read_csv(INPUT_FILE)
print("Total rows:", len(df))
# df.head(10)


Total rows: 92622


In [18]:
# 🧩 Cell 3: Define Validation Function
def validate_row(row):
    errors = []

    # 1. input_url
    if not str(row.get("input_url", "")).strip():
        errors.append("input_url blank")

    # 2. time_stamp
    if not str(row.get("time_stamp", "")).strip():
        errors.append("time_stamp blank")

    # 3. fb_url
    fb_url = str(row.get("fb_url", "")).strip()
    if not fb_url:
        errors.append("fb_url blank")
    elif "category" in fb_url.lower():
        errors.append("fb_url contains 'category'")
    elif fb_url.startswith("https://l.facebook.com/l.php"):
        errors.append("fb_url invalid redirect")

    # 4. fb_company_name
    if not str(row.get("fb_company_name", "")).strip():
        errors.append("fb_company_name blank")

    # 5. fb_email_address
    email = str(row.get("fb_email_address", "")).strip()
    if email and not is_valid_email(email):
        errors.append("invalid email_address")
    elif not email:
        errors.append("fb_email_address blank")

    # 6. fb_url_type
    fb_type = str(row.get("fb_url_type", "")).strip().lower()
    if fb_type not in ["official page", "unofficial page", "official", "unofficial"]:
        errors.append("fb_url_type invalid")

    # 7. fb_number_of_followers
    followers = str(row.get("fb_number_of_followers", "")).strip()
    if followers == "":
        errors.append("fb_number_of_followers blank")

    # 8. fb_phone_number
    phone = str(row.get("fb_phone_number", "")).strip()
    if not phone or not is_valid_phone(phone):
        errors.append("invalid fb_phone_number")

    return errors


In [19]:
# 🧩 Cell 4: Apply Validation Rules
valid_rows = []
invalid_rows = []

for _, row in df.iterrows():
    record = row.to_dict()
    issues = validate_row(record)

    if not issues:
        valid_rows.append(record)
    else:
        record["validation_errors"] = "; ".join(issues)
        invalid_rows.append(record)

valid_df = pd.DataFrame(valid_rows)
invalid_df = pd.DataFrame(invalid_rows)

print(f"✅ Total Valid: {len(valid_df)} | ❌ Total Invalid: {len(invalid_df)}")


✅ Total Valid: 43792 | ❌ Total Invalid: 48830


In [ ]:
# 🧩 Cell 5: Preview Top Rows
print("✅ VALID ROWS (Top 10):")
display(valid_df.head(10))

print("\n❌ INVALID ROWS (Top 10):")
display(invalid_df.head(10))


✅ VALID ROWS (Top 10):

❌ INVALID ROWS (Top 10):


In [12]:
# 🧩 Cell 6: (Optional) Save to CSVs for later comparison
valid_df.to_csv(OUTPUT_VALID, index=False)
invalid_df.to_csv(OUTPUT_INVALID, index=False)

print(f"Files saved → {OUTPUT_VALID}, {OUTPUT_INVALID}")


Files saved → /home/veronica/Desktop/dod-unprocessed-data/process_file.csv, /home/veronica/Desktop/dod-unprocessed-data/invalid_rows.csv
